In [567]:
import json
import pandas as pd

In [166]:
land_df = pd.read_csv('land_area.csv')
popn_df = pd.read_csv('population.csv')

In [167]:
land_df.head()

,Areaname,STCOU,LND010190F,LND010190D,LND010190N1,LND010190N2,LND010200F,LND010200D,LND010200N1,LND010200N2,LND110180F,LND110180D,LND110180N1,LND110180N2,LND110190F,LND110190D,LND110190N1,LND110190N2,LND110200F,LND110200D,LND110200N1,LND110200N2,LND110210F,LND110210D,LND110210N1,LND110210N2,LND210190F,LND210190D,LND210190N1,LND210190N2,LND210200F,LND210200D,LND210200N1,LND210200N2
0,UNITED STATES,0,0,3787425.08,0,0,0,3794083.06,0,0,0,3539289.16,0,0,0,3536341.73,0,0,0,3537438.44,0,0,0,3531905.43,0,0,0,251083.35,0,0,0,256644.62,0,0
1,ALABAMA,1000,0,52422.94,0,0,0,52419.02,0,0,0,50767.18,0,0,0,50750.23,0,0,0,50744.00,0,0,0,50645.33,0,0,0,1672.71,0,0,0,1675.01,0,0
2,"Autauga, AL",1001,0,604.49,0,0,0,604.45,0,0,0,597.04,0,0,0,596.01,0,0,0,595.97,0,0,0,594.44,0,0,0,8.48,0,0,0,8.48,0,0
3,"Baldwin, AL",1003,0,2027.08,0,0,0,2026.93,0,0,0,1589.42,0,0,0,1596.53,0,0,0,1596.35,0,0,0,1589.78,0,0,0,430.55,0,0,0,430.58,0,0
4,"Barbour, AL",1005,0,904.59,0,0,0,904.52,0,0,0,883.89,0,0,0,885.00,0,0,0,884.90,0,0,0,884.88,0,0,0,19.59,0,0,0,19.61,0,0


In [168]:
popn_df.head()

,Areaname,STCOU,PST045200F,PST045200D,PST045200N1,PST045200N2,PST045201F,PST045201D,PST045201N1,PST045201N2,PST045202F,PST045202D,PST045202N1,PST045202N2,PST045203F,PST045203D,PST045203N1,PST045203N2,PST045204F,PST045204D,PST045204N1,PST045204N2,PST045205F,PST045205D,PST045205N1,PST045205N2,PST045206F,PST045206D,PST045206N1,PST045206N2,PST045207F,PST045207D,PST045207N1,PST045207N2,PST045208F,PST045208D,PST045208N1,PST045208N2,PST045209F,PST045209D,PST045209N1,PST045209N2
0,UNITED STATES,0,0,282171957,0,0,0,285081556,0,0,0,287803914,0,0,0,290326418,0,0,0,293045739,0,0,0,295753151,0,0,0,298593212,0,0,0,301579895,0,0,0,304374846,0,0,0,307006550,0,0
1,ALABAMA,1000,0,4451849,0,0,0,4464034,0,0,0,4472420,0,0,0,4490591,0,0,0,4512190,0,0,0,4545049,0,0,0,4597688,0,0,0,4637904,0,0,0,4677464,0,0,0,4708708,0,0
2,"Autauga, AL",1001,0,43872,0,0,0,44434,0,0,0,45157,0,0,0,45762,0,0,0,46933,0,0,0,47870,0,0,0,49105,0,0,0,49834,0,0,0,50354,0,0,0,50756,0,0
3,"Baldwin, AL",1003,0,141358,0,0,0,144988,0,0,0,148141,0,0,0,151707,0,0,0,156573,0,0,0,162564,0,0,0,168516,0,0,0,172815,0,0,0,176212,0,0,0,179878,0,0
4,"Barbour, AL",1005,0,29035,0,0,0,29223,0,0,0,29289,0,0,0,29480,0,0,0,29458,0,0,0,29452,0,0,0,29556,0,0,0,29736,0,0,0,29836,0,0,0,29737,0,0


In [220]:
land_df = land_df[['Areaname', 'LND010190D']]
popn_df = popn_df[['Areaname', 'PST045200D']]

In [269]:
lower_48 = ["AL", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
            "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
            "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
            "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
            "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

state_end = tuple(', ' + abbrev for abbrev in lower_48)

In [270]:
filtered_land_df = land_df[land_df.Areaname.str.endswith(state_end)]
filtered_popn_df = popn_df[land_df.Areaname.str.endswith(state_end)]

In [271]:
filtered_popn_df.shape

(3111, 2)

In [272]:
virginia_counties_df = filtered_land_df[filtered_land_df.Areaname.str.endswith(', VA')]

In [273]:
indices_to_delete = []
counties_set = set()
for index, row in virginia_counties_df.iterrows():
    county = row['Areaname']
    if county not in counties_set:
        counties_set.add(county)
    else:
        indices_to_delete.append(index)

In [274]:
filtered_land_df = filtered_land_df[~filtered_land_df.index.isin(indices_to_delete)]
filtered_popn_df = filtered_popn_df[~filtered_popn_df.index.isin(indices_to_delete)]

In [275]:
len(filtered_popn_df)

3106

In [276]:
combined_df = pd.merge(filtered_land_df, filtered_popn_df, on='Areaname', how='inner')

In [277]:
combined_df.shape

(3106, 3)

In [278]:
combined_df['state'] = combined_df.Areaname.str[-2:]

In [279]:
combined_df[combined_df.state=='VA']

,Areaname,LND010190D,PST045200D,state
2786,"Accomack, VA",1310.18,38365,VA
2787,"Albemarle, VA",726.24,84622,VA
2788,"Alleghany, VA",446.93,17200,VA
2789,"Amelia, VA",358.57,11478,VA
2790,"Amherst, VA",478.81,31882,VA
2791,"Appomattox, VA",334.75,13691,VA
2792,"Arlington, VA",25.97,189527,VA
2793,"Augusta, VA",972.28,65772,VA
2794,"Bath, VA",534.59,5032,VA
2795,"Bedford, VA",769.36,60606,VA


In [280]:
combined_df.head()

,Areaname,LND010190D,PST045200D,state
0,"Autauga, AL",604.49,43872,AL
1,"Baldwin, AL",2027.08,141358,AL
2,"Barbour, AL",904.59,29035,AL
3,"Bibb, AL",625.50,19936,AL
4,"Blount, AL",650.65,51181,AL


In [281]:
combined_df.rename(columns={'Areaname': 'county', 'LND010190D': 'area', 'PST045200D': 'popn'}, inplace=True)

In [282]:
combined_df.loc[combined_df.county == 'Broomfield, CO', 'area'] = 33.00

In [283]:
combined_df['density'] = combined_df['popn'] / combined_df['area']

In [284]:
state2pop = combined_df.groupby('state').agg({'popn': sum}).to_dict()['popn']

In [285]:
combined_df['state_popn'] = [state2pop[state] for state in combined_df.state]

In [505]:
combined_df.head()

,county,area,popn,state,density,state_popn
0,"Autauga, AL",604.49,43872,AL,72.576883,4451849
1,"Baldwin, AL",2027.08,141358,AL,69.734791,4451849
2,"Barbour, AL",904.59,29035,AL,32.097414,4451849
3,"Bibb, AL",625.50,19936,AL,31.872102,4451849
4,"Blount, AL",650.65,51181,AL,78.661339,4451849


In [537]:
state2density_metric = (combined_df.groupby('state').
                        apply(lambda x: round(x['popn'] * (x['density'] ** 1) / x['state_popn'], 1))
                        .groupby('state').sum()).to_dict()

In [538]:
sorted_density_metrics = sorted(list(state2density_metric.values()), reverse=True)

In [539]:
density_metric2state = {v: k for k, v in state2density_metric.items()}

In [542]:
ordered_density_metric2state = {x: density_metric2state[x] for x in sorted_density_metrics}

In [544]:
state_stats_df = pd.DataFrame(ordered_density_metric2state.keys(), columns=['density_metric'], 
                              index=ordered_density_metric2state.values())

In [545]:
state_stats_df.head()

,density_metric
NY,10711.4
NJ,2789.6
PA,1957.6
IL,1761.9
MD,1737.6


In [546]:
latlong_df = pd.read_csv('statelatlong.csv')

In [547]:
latlong_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [548]:
state_stats_df1 = (pd.merge(state_stats_df, latlong_df[['Latitude', 'State']],
                           left_index=True, right_on='State').drop(columns=['State']))
state_stats_df1.index = ordered_density_metric2state.values()

In [549]:
coastal_states = set('ME NH MA RI CT NY NJ PA MD DE VA NC SC GA FL WA OR CA'.split())

In [550]:
state_stats_df1['is_coastal'] = [state in coastal_states for state in state_stats_df.index]

In [551]:
state2passengers = {'NY': 50868391, 
                    'PA': 15285948 + 4670954 + 636916, 
                    'NJ': 19923009 + 589091,
                    'MD': 13371816,
                    'IL': round((83245472 / 2) + (22027737 / 2)),
                    'MA': 17759044,
                    'VA': 11470854 + 10596942 + 1777648 + 1602631,
                    'MO': 6793076 + 5391557 + 462126,
                    'CA': (39636042 + 25707101 + 10340164 + 5934639 + 5321603 + 5217242 
                           + 4969366 + 2104625 + 2077892 + 1386357 + 995801 + 761298),
                    'MI': 16847135 + 1334979 + 398508,
                    'CO': 28267394 + 657694,
                    'MN': 18123844,
                    'TX': 31283579 + 20062072 + 7554596 + 6285181 + 6095545 + 4179994 + 1414376,
                    'RI': 1803000,
                    'GA': 50501858 + 1056265,
                    'OH': 4083476 + 3567864 + 1019922 + 685553,
                    'CT': 2982194,
                    'IN': 4216766 + 360369 + 329957 + 204352,
                    'DE': 0,
                    'KY': 3269979 + 1631494 + 638316,
                    'FL': (20875813 + 20283541 + 14263270 + 9194994 + 4239261 + 3100624 + 2729129 
                           + 1321675 + 986766 + 915672 + 589860),
                    'NE': 2127387 + 162876,
                    'UT': 11143738,
                    'OR': 9071154,
                    'TN': 6338517 + 2016089 + 887103,
                    'LA': 5569705 + 364200,
                    'OK': 1796473 + 1342315,
                    'NC': 21511880 + 5401714 + 848261,
                    'KS': 781944,
                    'WA': 21887110 + 1570652,
                    'WI': 3496724 + 1043185 + 348026 + 314909,
                    'NH': 995403,
                    'AL': 1304467 + 527801 + 288209 + 173210,
                    'NM': 2341719,
                    'IA': 1216357 + 547786,
                    'AZ': 20896265 + 1594594 + 705731,
                    'SC': 1811695 + 991276 + 944849 + 553658,
                    'AR': 958824 + 673810,
                    'WV': 213412,
                    'ID': 1633507,
                    'NV': 22833267 + 1771864,
                    'ME': 886343 + 269013,
                    'MS': 491464 + 305157,
                    'VT': 593311,
                    'SD': 510105 + 272537,
                    'ND': 402976 + 273980 + 150634 + 132557 + 68829,
                    'MT': 553245 + 423213 + 381582 + 247816 + 176730 + 103239,
                    'WY': 342044 + 92805}


In [552]:
state_stats_df1['airport_arrivals'] = [state2passengers[state] for state in state_stats_df.index]

In [553]:
state_stats_df1.head()

,density_metric,Latitude,is_coastal,airport_arrivals
NY,10711.4,40.705626,True,50868391
NJ,2789.6,40.143006,True,20512100
PA,1957.6,40.994593,True,20593818
IL,1761.9,39.739318,False,52636604
MD,1737.6,38.806352,True,13371816


In [569]:
state_to_neighbors = {'AL': {'MS', 'TN', 'FL', 'GA', 'NC', 'SC'},
                      'GA': {'TN', 'FL', 'AL', 'SC', 'NC', 'MS'},
                      'FL': {'GA', 'AL', 'MS', 'SC'},
                      'MS': {'AL', 'TN', 'FL', 'LA', 'AR', 'GA'},
                      'LA': {'TX', 'AR', 'MS', 'OK', 'AL'},
                      'SC': {'FL', 'GA', 'NC', 'TN'},
                      'NC': {'SC', 'GA', 'TN', 'VA', 'KY'},
                      'AR': {'LA', 'TX', 'MS', 'TN', 'OK', 'MO', 'KY'},
                      'VA': {'NC', 'KY', 'WV', 'TN', 'DC', 'MD', 'DE'},
                      'DC': {'VA', 'MD', 'DE'},
                      'MD': {'DC', 'VA', 'WV', 'DE', 'NJ', 'PA'},
                      'DE': {'MD', 'DC', 'NJ', 'PA'},
                      'NJ': {'DE', 'MD', 'PA', 'NY', 'NJ', 'CT'},
                      'NY': {'NJ', 'PA', 'CT', 'MA', 'VT'},
                      'CT': {'NY', 'RI', 'MA'},
                      'RI': {'CT', 'MA'},
                      'MA': {'CT', 'RI', 'NH', 'VT', 'NY'},
                      'NH': {'VT', 'ME', 'MA'},
                      'ME': {'NH', 'MA', 'VT'},
                      'VT': {'NH', 'NY', 'MA'},
                      'PA': {'NY', 'NJ', 'MD', 'WV', 'OH', 'DE'},
                      'WV': {'DC', 'MD', 'PA', 'OH', 'KY', 'VA'},
                      'OH': {'PA', 'WV', 'MI', 'IN', 'KY'},
                      'MI': {'OH', 'WI', 'IN', 'IL'},
                      'KY': {'WV', 'OH', 'IN', 'IL', 'MO', 'TN', 'VA', 'AR', 'NC'},
                      'TN': {'KY', 'VA', 'NC', 'SC', 'GA', 'AL', 'MS', 'AR', 'MO', 'IL'},
                      'IN': {'KY', 'OH', 'MI', 'IL', 'WI'},
                      'IL': {'IN', 'MI', 'WI', 'IA', 'MO', 'KY', 'TN'},
                      'WI': {'IL', 'MN', 'MI', 'IA'},
                      'MN': {'MI', 'WI', 'IA', 'ND', 'SD', 'NE', 'IL'},
                      'IA': {'WI', 'MN', 'IL', 'MO', 'KS', 'NE', 'SD'},
                      'MO': {'IA', 'IL', 'KY', 'TN', 'AR', 'OK', 'KS', 'NE'},
                      'ND': {'SD', 'MN', 'MT', 'WY'},
                      'SD': {'ND', 'MN', 'IA', 'NE', 'MT', 'WY'},
                      'NE': {'SD', 'IA', 'MO', 'KS', 'WY', 'CO'},
                      'KS': {'NE', 'IA', 'MO', 'AR', 'OK', 'CO', 'TX', 'NM'},
                      'OK': {'KS', 'MO', 'AR', 'TX', 'NM', 'CO', 'LA'},
                      'TX': {'LA', 'AR', 'OK', 'NM', 'CO'},
                      'MT': {'ND', 'SD', 'WY', 'ID'},
                      'WY': {'MT', 'ND', 'SD', 'NE', 'CO', 'UT', 'ID'},
                      'CO': {'WY', 'NE', 'KS', 'OK', 'TX', 'NM', 'UT', 'AZ'},
                      'NM': {'CO', 'KS', 'OK', 'TX', 'AZ', 'UT'},
                      'ID': {'MT', 'WY', 'UT', 'NV', 'WA', 'OR'},
                      'UT': {'ID', 'WY', 'CO', 'NM', 'AZ', 'NV'},
                      'AZ': {'NM', 'CO', 'UT', 'NV', 'CA'},
                      'WA': {'ID', 'OR'},
                      'OR': {'WA', 'ID', 'NV', 'CA'},
                      'NV': {'ID', 'OR', 'UT', 'AZ', 'CA'},
                      'CA': {'OR', 'NV', 'AZ'},
                      }

state_to_neighbors = {k: list(v) for k, v in state_to_neighbors.items()}

In [555]:
age_df = pd.read_csv('age.csv')

In [556]:
abbrev2state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [557]:
state2abbrev = {v: k for k, v in abbrev2state.items()}

In [558]:
age_df['Location'] = [state2abbrev[state] for state in age_df.Location]

In [559]:
state_stats_df2 = (pd.merge(state_stats_df1, age_df, left_index=True, right_on='Location')
                  .drop(columns=['Location']))
state_stats_df2.index = ordered_density_metric2state.values()

In [560]:
state_stats_df2.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+
NY,10711.4,40.705626,True,50868391,0.22,0.09,0.13,0.26,0.14,0.16
NJ,2789.6,40.143006,True,20512100,0.23,0.08,0.11,0.27,0.14,0.16
PA,1957.6,40.994593,True,20593818,0.22,0.08,0.12,0.25,0.14,0.18
IL,1761.9,39.739318,False,52636604,0.24,0.09,0.12,0.26,0.13,0.15
MD,1737.6,38.806352,True,13371816,0.23,0.08,0.12,0.27,0.14,0.15


In [561]:
temps_df = pd.read_csv('temps.csv')

In [562]:
temps_df['State'] = [state2abbrev[state] for state in temps_df.State]

In [563]:
state_stats_df3 = (pd.merge(state_stats_df2, temps_df, left_index=True, right_on='State')
                  .drop(columns=['State']))
state_stats_df3.index = ordered_density_metric2state.values()

In [564]:
state_stats_df3.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,spring,summer,fall,winter
NY,10711.4,40.705626,True,50868391,0.22,0.09,0.13,0.26,0.14,0.16,43.6,66.5,48.1,23.3
NJ,2789.6,40.143006,True,20512100,0.23,0.08,0.11,0.27,0.14,0.16,50.6,72.2,54.8,33.0
PA,1957.6,40.994593,True,20593818,0.22,0.08,0.12,0.25,0.14,0.18,47.4,68.6,50.9,28.4
IL,1761.9,39.739318,False,52636604,0.24,0.09,0.12,0.26,0.13,0.15,51.6,73.4,53.8,28.3
MD,1737.6,38.806352,True,13371816,0.23,0.08,0.12,0.27,0.14,0.15,52.8,73.3,56.1,34.7


In [566]:
state_stats_df3.to_csv('state_stats.csv')

In [570]:
with open('state2neighbors.csv', 'w') as f:
    f.write(json.dumps(state_to_neighbors))